## This notebook is for the analysis of 15 16S PCR libraries that were produced from the swabs that the astronaut used to samples the ISS as a part of Project MERCCURI.</p></p>  Before launching this ipython notebook, I typed the macqiime command to configure the shell. I'm using macqiime 1.8.0 http://www.wernerlab.org/software/macqiime/macqiime-installation  

#### This workflow depends on the presence of 4 files.</p></p>1) demultiplexed sequences</p>2) a mapping file</p>3) a perlscript (code is below)</p>4) a parameter file (parameters.txt) with the following 2 lines:</p>pick_otus:enable_rev_strand_match       True</p> make_emperor:ignore_missing_samples	True</p>To run a different project through this workflow:</p></p>1) do a search and replace for SpaceSwabs.</p>2) change the -e value for the core diversity analyses.

#### I copied the stuff below from a QIIME/iPython notebook tutorial: http://nbviewer.ipython.org/github/qiime/qiime/blob/1.8.0/examples/ipynb/illumina_overview_tutorial.ipynb</p></p>I'm not sure what all of it does...

In [1]:
from os import chdir, mkdir
from os.path import join
#the following are only available in the current development branch of IPython
from IPython.display import FileLinks, FileLink

#### The version of macqiime that I'm using does not install the greengenes 99% cutoff OTUs and taxonomy, so did that manually as per the instructions on the MacQIIME Installation site. I just substituted the gg_13_8_otus folder that has all of the otu cutoffs for the one included in macqiime/greengenes/ 

In [2]:
mapping_file = "SpaceSwabs.txt"
otu_base = "/macqiime/greengenes/gg_13_8_otus/"
reference_seqs_99 = join(otu_base,"/macqiime/greengenes/gg_13_8_otus/rep_set/99_otus.fasta")
reference_tree_99 = join(otu_base,"/macqiime/greengenes/gg_13_8_otus/trees/99_otus.tree")
reference_tax_99 = join(otu_base,"/macqiime/greengenes/gg_13_8_otus/taxonomy/99_otu_taxonomy.txt")
reference_seqs_97 = join(otu_base,"/macqiime/greengenes/gg_13_8_otus/rep_set/97_otus.fasta")
reference_tree_97 = join(otu_base,"/macqiime/greengenes/gg_13_8_otus/trees/97_otus.tree")
reference_tax_97 = join(otu_base,"/macqiime/greengenes/gg_13_8_otus/taxonomy/97_otu_taxonomy.txt")
pynast_ref_alignment = join(otu_base, "/macqiime/QIIME/qiime_test_data/identify_chimeric_seqs/ref_seq_set_aligned.fasta")

### make some directories to hold the output

In [57]:
!mkdir sequence_data
!mkdir OTU_picking
!mkdir core_diversity
!mkdir biom_tables
!mkdir biom_summaries
!mkdir misc_files

### Make sure mapping file is good to go. This also provides a quick check for macqiime.

In [4]:
!validate_mapping_file.py -m $mapping_file
!mv SpaceSwabs_corrected.txt SpaceSwabs.txt

Errors and/or warnings detected in mapping file.  Please check the log and html file for details.


#### I have this line in my parameter file that makes the following step unecessary, but I do this anyway because our demultiplexing script returns all of the merged sequences in the RC orientation. Flipping them around before doing the OTU-picking makes it run MUCH faster. </p></p>pick_otus:enable_rev_strand_match       True

In [58]:
!adjust_seq_orientation.py -i SpaceSwabs.fasta -o sequence_data/SpaceSwabs.faa
!rm SpaceSwabs.fasta

# Pick OTUs

#### Because I'm not sure what I will need downstream, I just pick both open- and closed-reference OTUs, clustered at both 97% and 99% similarity. </p></p>The pick_open_reference_otus.py script will cluster all of the sequences, assign taxonomy to the OTUs (when possible, a greengenes ID will be assigned,) choose a representative sequence from each OTU (rep_set), and align and build a phylogenetic tree from the representative sequences. OTUs that do not align to PyNast and singleton OTUs are not included in this biom file (otu_table_mc2_w_tax_no_pynast_failures.biom.)

In [5]:
!pick_open_reference_otus.py -p parameters.txt -f -r $reference_seqs_97 -i sequence_data/SpaceSwabs.faa -o OTU_picking/SpaceSwabs_97_open_reference_otus -n ISS -a -O 4 
!cp OTU_picking/SpaceSwabs_97_open_reference_otus/otu_table_mc2_w_tax_no_pynast_failures.biom biom_tables/97_open.biom

In [6]:
!pick_open_reference_otus.py -p parameters.txt -f -r $reference_seqs_99 -i sequence_data/SpaceSwabs.faa -o OTU_picking/SpaceSwabs_99_open_reference_otus -n ISS -a -O 4 
!cp OTU_picking/SpaceSwabs_97_open_reference_otus/otu_table_mc2_w_tax_no_pynast_failures.biom biom_tables/99_open.biom

In [1]:
!ls

97_closed.list                  group_significance.L3.nodes
97_closed_700_tax.list          group_significance.L4.human
97_closed_pathos.txt            group_significance.L4.nodes
Figures                         group_significance.L4.nodes.001
OTU_picking                     group_significance.L4.nodes.01
Rplot01.pdf                     group_significance.L4.nodes.1
Rplot05.pdf                     group_significance.L4.touches
Rplot06.pdf                     group_significance.nodes
Rplot07.pdf                     heatmap_all_taxa.svg
SpaceSwabs.csv                  js
SpaceSwabs.faa                  misc_files
SpaceSwabs.ipynb                open97
SpaceSwabs.no2.txt              open99
SpaceSwabs.txt                  parameters.txt
area_charts.html                pathoID.list
beta_div_bray                   pathos.list
biom_summaries                  pd.txt
biom_tables                     pie_charts.html
biplot_options                  raw_data
charts                          sequenc

#### The pick_closed_reference_otus.py script will cluster sequences against a reference (greengenes, here) database. Anything that does not hit a sequence in the database will be excluded from the resultant biom file. No tree is generated here, downstream analyses that depend on one must use a reference (greengenes, here) phylogeny.

In [25]:
!pick_closed_reference_otus.py -p parameters.txt -f -r $reference_seqs_97 -t $reference_tax_97 -i sequence_data/SpaceSwabs.faa -o OTU_picking/SpaceSwabs_97_closed_reference_otus -a -O 4 
!cp OTU_picking/SpaceSwabs_97_closed_reference_otus/SpaceSwabs_otu_table.biom biom_tables/97_closed.biom

## dddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddvvvvvvvvvvvvvbva ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ARG!cp OTU_picking/SpaceSwabs_99_closed_reference_otus/SpaceSwabs_otu_table.biom biom_tables/99_closed.biom

### Remove chloroplast, mitochondria, and Unassigned at the domain level (in my experience, these are usually chloroplast, but one can check with a blast search.)

In [9]:
!filter_taxa_from_otu_table.py -i biom_tables/97_open.biom -o biom_tables/97_open_filtered.biom -n c__Chloroplast,f__mitochondria,Unassigned

In [10]:
!filter_taxa_from_otu_table.py -i biom_tables/99_open.biom -o biom_tables/99_open_filtered.biom -n c__Chloroplast,f__mitochondria,Unassigned

In [27]:
!filter_taxa_from_otu_table.py -i biom_tables/97_closed.biom -o biom_tables/97_closed_filtered.biom -n c__Chloroplast,f__mitochondria,Unassigned

In [28]:
!filter_taxa_from_otu_table.py -i biom_tables/99_closed.biom -o biom_tables/99_closed_filtered.biom -n c__Chloroplast,f__mitochondria,Unassigned

### For open-reference-picked OTUs: Identify and remove chimeric sequences. Not a concern with closed-reference OTU-picking, because sequences have to match something in the reference database.

#### Identify chimeras

In [13]:
!identify_chimeric_seqs.py -m usearch61

Error in identify_chimeric_seqs.py: Required option --input_fasta_fp omitted.

If you need help with QIIME, see:
http://help.qiime.org


In [29]:
!identify_chimeric_seqs.py -m ChimeraSlayer -i OTU_picking/SpaceSwabs_97_open_reference_otus/pynast_aligned_seqs/rep_set_aligned.fasta -a $pynast_ref_alignment -o 97_open_chimeras.txt

In [30]:
!identify_chimeric_seqs.py -m ChimeraSlayer -i OTU_picking/SpaceSwabs_99_open_reference_otus/pynast_aligned_seqs/rep_set_aligned.fasta -a $pynast_ref_alignment -o 99_open_chimeras.txt

#### Remove chimeras from OTU table

In [35]:
!filter_otus_from_otu_table.py -i biom_tables/97_open_filtered.biom -o biom_tables/97_open_filtered_no_chimeras.biom -e 97_open_chimeras.txt

In [36]:
!filter_otus_from_otu_table.py -i biom_tables/99_open_filtered.biom -o biom_tables/99_open_filtered_no_chimeras.biom -e 99_open_chimeras.txt

### Sanity check for the otu tables. Make a note of the values to be used for rarefaction.

In [7]:
!per_library_stats.py -i biom_tables/97_open_filtered.biom --num_otus

/bin/sh: per_library_stats.py: command not found


In [11]:
!rm tmp
!biom summarize-table -o tmp -i biom_tables/97_open.biom --qualitative
!cat tmp

rm: tmp: No such file or directory
Num samples: 15
Num observations: 13077

Observations/sample summary:
 Min: 1036
 Max: 4294
 Median: 1757.000
 Mean: 2049.867
 Std. dev.: 806.243
 Sample Metadata Categories: None provided
 Observation Metadata Categories: taxonomy

Observations/sample detail:
 SP9: 1036
 SP2: 1320
 SP11: 1349
 SP12: 1429
 SP7: 1456
 SP13: 1678
 SP1: 1744
 SP10: 1757
 SP5: 1995
 SP6: 2129
 SP15: 2380
 SP3: 2457
 SP4: 2820
 SP14: 2904
 SP8: 4294


In [33]:
#!biom summarize-table -i biom_tables/97_open_filtered_no_chimeras.biom -o biom_summaries/97_open_filtered_no_chimeras.summary
!cat biom_summaries/97_open_filtered_no_chimeras.summary #rarefy to: 

Num samples: 15
Num observations: 12554
Total count: 857296
Table density (fraction of non-zero values): 0.159
Table md5 (unzipped): 72f04b83b6a1a9a3c4f4b10eeb86755c

Counts/sample summary:
 Min: 26221.0
 Max: 76656.0
 Median: 60537.000
 Mean: 57153.067
 Std. dev.: 14564.147
 Sample Metadata Categories: None provided
 Observation Metadata Categories: taxonomy

Counts/sample detail:
 SP9: 26221.0
 SP2: 29887.0
 SP1: 43573.0
 SP12: 49621.0
 SP10: 49947.0
 SP15: 56290.0
 SP11: 59525.0
 SP3: 60537.0
 SP13: 61854.0
 SP8: 62778.0
 SP7: 64454.0
 SP14: 68636.0
 SP6: 71861.0
 SP4: 75456.0
 SP5: 76656.0


In [19]:
!biom summarize-table -i biom_tables/99_open_filtered_no_chimeras.biom -o biom_summaries/99_open_filtered_no_chimeras.summary
!cat biom_summaries/99_open_filtered_no_chimeras.summary #rarefy to: 

In [21]:
!biom summarize-table -i biom_tables/97_closed_filtered.biom -o biom_summaries/97_closed_filtered.summary
!cat biom_summaries/97_closed_filtered.summary #rarefy to: 

In [23]:
!biom summarize-table -i biom_tables/99_closed_filtered.biom -o biom_summaries/99_closed_filtered.summary
!cat biom_summaries/99_closed_filtered.summary #rarefy to: 

### During closed-reference OTU-picking, neither an alignment nor a tree of the representative sequences is produced.  This is because QIIME will just use the full greengenes phylogeny if it needs one. But, if I want to use something else like phyloseq, then I want to have a phylogeny of the OTUs in my samples, and not the full greengenes tree, and I think this is the easiest way to get one. I'm making a new tree for the open-reference OTUs as well, because I filtered out chimeras, chloroplasts, etc.

#### First, convert the biom tables into classic format so that I can easily get the OTU IDs

In [35]:
!biom convert -b -i biom_tables/97_closed_filtered.biom -o biom_tables/97_closed_filtered.txt

In [36]:
!biom convert -b -i biom_tables/99_closed_filtered.biom -o biom_tables/99_closed_filtered.txt

In [40]:
!biom convert -b -i biom_tables/97_open_filtered_no_chimeras.biom -o biom_tables/97_open_filtered_no_chimeras.txt

In [39]:
!biom convert -b -i biom_tables/99_open_filtered_no_chimeras.biom -o biom_tables/99_open_filtered_no_chimeras.txt

#### Use this stupid perl script to make a list of OTU IDs for each biom table.

#### Extract an alignment of the taxa in my OTU table from the greengenes alignment for closed-reference, and from the repset alignment for open-reference..

In [49]:
!filter_fasta.py -f /macqiime/greengenes/gg_13_8_otus/rep_set_aligned/97_otus.fasta -o 97_closed_aligned.fasta -s 97_closed.list
!mv 97_closed.list misc_files

In [73]:
!filter_fasta.py -f /macqiime/greengenes/gg_13_8_otus/rep_set_aligned/99_otus.fasta -o 99_closed_aligned.fasta -s 99_closed.list
!mv 99_closed.list misc_files

In [64]:
!filter_fasta.py -f OTU_picking/SpaceSwabs_97_open_reference_otus/new_refseqs.fna -o 97_open_aligned.fasta -s 97_open.list
!mv 97_open.list misc_files

In [65]:
!filter_fasta.py -f OTU_picking/SpaceSwabs_97_open_reference_otus/new_refseqs.fna -o 99_open_aligned.fasta -s 99_open.list
!mv 99_open.list misc_files

#### Build a tree from that alignment

In [53]:
!make_phylogeny.py -i 97_open_aligned.fasta

In [54]:
!make_phylogeny.py -i 99_open_aligned.fasta

In [66]:
!make_phylogeny.py -i 97_closed_aligned.fasta

In [76]:
!make_phylogeny.py -i 99_closed_aligned.fasta

## Now, for all 4 datasets (open97, open99, closed97, closed99) I have a biom table, an alignment, and a phylogenetic tree. Now, clean up the directory.

In [68]:
!mkdir open97
!cp biom_tables/97_open_filtered_no_chimeras.biom open97/SpaceSwabs_open97.biom
!mv 97_open_aligned.fasta open97/SpaceSwabs_open97.fasta
!mv 97_open_aligned.tre open97/SpaceSwabs_open97.tre

In [69]:
!mkdir open99
!cp biom_tables/99_open_filtered_no_chimeras.biom open99/SpaceSwabs_open99.biom
!mv 99_open_aligned.fasta open99/SpaceSwabs_open99.fasta
!mv 99_open_aligned.tre open99/SpaceSwabs_open99.tre

In [70]:
!mkdir closed97
!cp biom_tables/97_closed_filtered.biom closed97/SpaceSwabs_closed97.biom
!mv 97_closed_aligned.fasta closed97/SpaceSwabs_closed97.fasta
!mv 97_closed_aligned.tre closed97/SpaceSwabs_closed97.tre

In [78]:
!mkdir closed99
!cp biom_tables/99_closed_filtered.biom closed99/SpaceSwabs_closed99.biom
!mv 99_closed_aligned.fasta closed99/SpaceSwabs_closed99.fasta
!mv 99_closed_aligned.tre closed99/SpaceSwabs_closed99.tre

In [81]:
!mv SpaceSwabs.html misc_files
!mv SpaceSwabs.log misc_files
!mv overlib.js misc_files/

# Core diversity analyses

In [11]:
!core_diversity_analyses.py -p parameters.txt -i open97/SpaceSwabs_open97.biom -o core_diversity/SpaceSwabs_open97 -m SpaceSwabs.txt -e 26221 -t open97/SpaceSwabs_open97.tre

In [12]:
!core_diversity_analyses.py -p parameters.txt -i open99/SpaceSwabs_open99.biom -o core_diversity/SpaceSwabs_open99 -m SpaceSwabs.txt -e 26221 -t open99/SpaceSwabs_open99.tre

In [13]:
!core_diversity_analyses.py -p parameters.txt -i closed97/SpaceSwabs_closed97.biom -o core_diversity/SpaceSwabs_closed97 -m SpaceSwabs.txt -e 25200 -t closed97/SpaceSwabs_closed97.tre

In [14]:
!core_diversity_analyses.py -p parameters.txt -i closed99/SpaceSwabs_closed99.biom -o core_diversity/SpaceSwabs_closed99 -m SpaceSwabs.txt -e 25486 -t closed99/SpaceSwabs_closed99.tre

# Add metadata for phinch

In [16]:
!biom add-metadata -i open97/SpaceSwabs_open97.biom -m SpaceSwabs.txt -o biom_tables/SpaceSwabs_open97_wmd.biom

In [ ]:
!biom add-metadata -i open99/SpaceSwabs_open99.biom -m SpaceSwabs.txt -o biom_tables/SpaceSwabs_open99_wmd.biom

In [ ]:
!biom add-metadata -i closed97/SpaceSwabs_closed97.biom -m SpaceSwabs.txt -o biom_tables/SpaceSwabs_closed97_wmd.biom

In [ ]:
!biom add-metadata -i closed99/SpaceSwabs_closed99.biom -m SpaceSwabs.txt -o biom_tables/SpaceSwabs_closed99_wmd.biom

# Additional analyses

#### Which taxa are differentially represented in lab vs. crew nodes? To answer this, I first removed the node2 sample from the mapping file, because there is only one of those.

In [5]:
!group_significance.py -i biom_tables/97_open_filtered_no_chimeras.biom -m SpaceSwabs.no2.txt -c node -s g_test -o group_significance.nodes --biom_samples_are_superset

#### There are many significant OTUs here, but the vast majority are from just a handful of classes, so I'll collapse the taxonomy and look again.

In [38]:
!summarize_taxa.py -i biom_tables/97_open_filtered_no_chimeras.biom -L 4 -o biom_tables/97_open_filtered_no_chimeras.L4
!summarize_taxa.py -i biom_tables/97_open_filtered_no_chimeras.biom -L 3 -o biom_tables/97_open_filtered_no_chimeras.L3 -a

In [41]:
!group_significance.py --metadata_key taxonomy -i biom_tables/97_open_filtered_no_chimeras.L4/97_open_filtered_no_chimeras_L4.biom -m SpaceSwabs.no2.txt -c node -s g_test -o group_significance.L4.nodes --biom_samples_are_superset
!group_significance.py --metadata_key taxonomy -i biom_tables/97_open_filtered_no_chimeras.L3/97_open_filtered_no_chimeras_L3.biom -m SpaceSwabs.no2.txt -c node -s g_test -o group_significance.L3.nodes --biom_samples_are_superset

No metadata in biom table.
No metadata in biom table.


In [46]:
!group_significance.py --metadata_key taxonomy -i biom_tables/97_open_filtered_no_chimeras.L4/97_open_filtered_no_chimeras_L4.biom -m SpaceSwabs.no2.txt -c touches -o group_significance.L4.touches --biom_samples_are_superset
!group_significance.py --metadata_key taxonomy -i biom_tables/97_open_filtered_no_chimeras.L4/97_open_filtered_no_chimeras_L4.biom -m SpaceSwabs.no2.txt -c human -o group_significance.L4.human --biom_samples_are_superset

No metadata in biom table.
No metadata in biom table.


#### "No metadata in biom table." is just a warning.

#### There are a lot of really low-abundance OTUs that differ significantly between nodes, so I'm going to filter the biom table and run this again.

In [30]:
!filter_otus_from_otu_table.py --min_count_fraction=.001 -i biom_tables/97_open_filtered_no_chimeras.biom -o biom_tables/97_open_filtered_no_chimeras.001

857296.0 857.296


In [36]:
#!rm biom_tables/97_open_filtered_no_chimeras.L4.1
!summarize_taxa.py -i biom_tables/97_open_filtered_no_chimeras.001 -L 4 -o biom_tables/97_open_filtered_no_chimeras.L4.001 -a

In [37]:
!group_significance.py --metadata_key taxonomy -i biom_tables/97_open_filtered_no_chimeras.L4.001/97_open_filtered_no_chimeras_L4.biom -m SpaceSwabs.no2.txt -c node -s g_test -o group_significance.L4.nodes.001 --biom_samples_are_superset

No metadata in biom table.
